In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [3]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [5]:
input_dim = 28       # Giriş boyutu (her zaman adımı 28 piksel)
hidden_dim = 128     # Gizli katman boyutu
layer_dim = 2        # 2 katmanlı LSTM
output_dim = 10      # 10 sınıf (0-9 rakamları)
sequence_length = 28 # Zaman adımı (her resim 28x28 olduğundan)
num_epochs = 5
batch_size = 100
learning_rate = 0.001  # Adam ile daha stabil öğrenme

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [9]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [11]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        
        # Katman ve Boyutları
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        
        # LSTM Katmanı
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        
        # Tam Bağlantılı (Fully Connected) Çıkış Katmanı
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Başlangıç gizli durumu ve hücre durumu (GPU için .to(device) ekledik)
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(device)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(device)

        # LSTM ileri yayılım (forward propagation)
        out, _ = self.lstm(x, (h0, c0))  # out: (batch_size, seq_length, hidden_dim)
        
        # Son zaman adımındaki çıktıyı al ve tam bağlı katmana ver
        out = self.fc(out[:, -1, :])  # Son zaman adımındaki çıktı
        return out

In [13]:
# Modeli Tanımlama
model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, sequence_length, input_dim).to(device)  # [batch_size, seq_length, input_size]
        labels = labels.to(device)

        # Gradients sıfırlama
        optimizer.zero_grad()

        # Forward propagation
        outputs = model(images)

        # Loss hesaplama
        loss = criterion(outputs, labels)

        # Backward propagation
        loss.backward()

        # Parametre güncelleme
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")


Epoch [1/5], Step [100/600], Loss: 0.7130
Epoch [1/5], Step [200/600], Loss: 0.2594
Epoch [1/5], Step [300/600], Loss: 0.2172
Epoch [1/5], Step [400/600], Loss: 0.0794
Epoch [1/5], Step [500/600], Loss: 0.2335
Epoch [1/5], Step [600/600], Loss: 0.1992
Epoch [2/5], Step [100/600], Loss: 0.0606
Epoch [2/5], Step [200/600], Loss: 0.1028
Epoch [2/5], Step [300/600], Loss: 0.0315
Epoch [2/5], Step [400/600], Loss: 0.1238
Epoch [2/5], Step [500/600], Loss: 0.1327
Epoch [2/5], Step [600/600], Loss: 0.0662
Epoch [3/5], Step [100/600], Loss: 0.0077
Epoch [3/5], Step [200/600], Loss: 0.0635
Epoch [3/5], Step [300/600], Loss: 0.0241
Epoch [3/5], Step [400/600], Loss: 0.0795
Epoch [3/5], Step [500/600], Loss: 0.0164
Epoch [3/5], Step [600/600], Loss: 0.0293
Epoch [4/5], Step [100/600], Loss: 0.0378
Epoch [4/5], Step [200/600], Loss: 0.0921
Epoch [4/5], Step [300/600], Loss: 0.0656
Epoch [4/5], Step [400/600], Loss: 0.0656
Epoch [4/5], Step [500/600], Loss: 0.0844
Epoch [4/5], Step [600/600], Loss:

In [17]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.view(-1, sequence_length, input_dim).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

print("Eğitim Tamamlandı!")

Test Accuracy: 98.40%
Eğitim Tamamlandı!
